## Backprojetion from triangulation

In [1]:
# import necessary modules
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

import multicam_calibration as mcc
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
import plotly.express as px
import pickle

In [3]:
from threed_utils.visualization import plot_triangulations, plot_back_projections
from threed_utils.old_triangulation import back_project_points
from utils import read_first_frame

### get triangulation from predictions: mcc, anipose

In [4]:
# get data from one session:
from pathlib import Path
dir_path = Path(r"D:\P05_3DRIG_YE-LP\e01_mouse_hunting\v04_mice-hunting\20240724\100455\multicam_video_2024-07-24T10_20_07_cropped_20241209165236")
calibration_path = Path(r"D:\P05_3DRIG_YE-LP\e01_mouse_hunting\v04_mice-hunting\20240724\calibration\multicam_video_2024-07-24T14_13_45_cropped_20241209165236")

In [ ]:
# get calibration data from 2_calibration_multicam_script.py on calibration_path (just change path inside) []

# store calibration data for backprojection


# use calibration data on 3_triangulation_mcc_anipose_script.py to obtain triangulation data for both


# plot data in 3d with the function


# modify backprojection to work with xarrays 

# once backprojection works, create function with plotting


# unify the two into a single script


# Resources: videos, calibration matrices, triangulation data

